In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Function to encode labels
def encode_labels(df, target_column, severity_column):
    le_target = LabelEncoder()
    le_severity = LabelEncoder()

    # Encode 'Target' and 'Severity' labels into integers
    df[target_column] = le_target.fit_transform(df[target_column])
    df[severity_column] = le_severity.fit_transform(df[severity_column])

    return df, le_target, le_severity

# Custom Dataset
class TargetSeverityDataset(Dataset):
    def __init__(self, texts, target_labels, severity_labels, tokenizer, max_len=128):
        self.texts = texts
        self.target_labels = target_labels
        self.severity_labels = severity_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_label = self.target_labels[idx]
        severity_label = self.severity_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'target_label': torch.tensor(target_label, dtype=torch.long),
            'severity_label': torch.tensor(severity_label, dtype=torch.long)
        }

# Multi-task Model with Softmax
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.4, num_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_classes)
        self.severity_classifier = nn.Linear(256, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        # Apply classifiers
        target_output = self.target_classifier(x)
        severity_output = self.severity_classifier(x)

        return target_output, severity_output


# Updated Training Function
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_labels = batch['target_label'].to(device)
            severity_labels = batch['severity_label'].to(device)

            optimizer.zero_grad()
            target_output, severity_output = model(input_ids, attention_mask)

            # Compute losses
            target_loss = criterion(target_output, target_labels)
            severity_loss = criterion(severity_output, severity_labels)
            total_batch_loss = target_loss + severity_loss

            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += total_batch_loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_target_preds, all_severity_preds = [], []
        all_target_labels, all_severity_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target_labels = batch['target_label'].to(device)
                severity_labels = batch['severity_label'].to(device)

                target_output, severity_output = model(input_ids, attention_mask)

                target_loss = criterion(target_output, target_labels)
                severity_loss = criterion(severity_output, severity_labels)
                val_loss += (target_loss + severity_loss).item()

                # Convert logits to predictions
                all_target_preds.extend(torch.argmax(target_output, dim=1).cpu().numpy())
                all_severity_preds.extend(torch.argmax(severity_output, dim=1).cpu().numpy())
                all_target_labels.extend(target_labels.cpu().numpy())
                all_severity_labels.extend(severity_labels.cpu().numpy())

        # Print epoch results
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        scheduler.step(val_loss / len(val_loader))

        print("Classification Report for Target Task:")
        print(classification_report(all_target_labels, all_target_preds))
        print("Classification Report for Severity Task:")
        print(classification_report(all_severity_labels, all_severity_preds))

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), r'/content/drive/MyDrive/best_model.pth')  # Save the best model
        # else:
        #     early_stop_counter += 1
        #     if early_stop_counter >= patience:
        #         print("Early stopping triggered")
        #         break

# Usage example
def main():
    # Load and preprocess your data
    df_train = pd.read_csv(r'/content/drive/MyDrive/B_trans.csv')  # Training data
    df_val = pd.read_csv(r'/content/drive/MyDrive/B_val_trans.csv')  # Validation data

    # Encode the labels
    df_train, le_target, le_severity = encode_labels(df_train, 'Target', 'Severity')
    df_val, _, _ = encode_labels(df_val, 'Target', 'Severity')  # Apply same encoding

    # Clean and preprocess the data
    df_train.dropna(subset=['Tweet'], inplace=True)
    df_val.dropna(subset=['Tweet'], inplace=True)

    # Extract texts and labels for training and validation
    texts_train = df_train['Tweet'].values
    target_labels_train = df_train['Target'].values
    severity_labels_train = df_train['Severity'].values

    texts_val = df_val['Tweet'].values
    target_labels_val = df_val['Target'].values
    severity_labels_val = df_val['Severity'].values

    # Initialize tokenizer and model for HateBERT
    tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
    bert_model = AutoModel.from_pretrained("GroNLP/hateBERT")

    # Create datasets
    train_dataset = TargetSeverityDataset(texts_train, target_labels_train, severity_labels_train, tokenizer)
    val_dataset = TargetSeverityDataset(texts_val, target_labels_val, severity_labels_val, tokenizer)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)

    # Train model
    train_model(model, train_loader, val_loader, device)

# Run the main function
if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch 1/30
Training Loss: 2.5948
Validation Loss: 2.5329
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       140
           1       0.41      0.95      0.58       274
           2       0.00      0.00      0.00        99
           3       0.57      0.34      0.42       286

    accuracy                           0.45       799
   macro avg       0.25      0.32      0.25       799
weighted avg       0.35      0.45      0.35       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        74
           1       0.33      0.80      0.47       257
           2       0.00      0.00      0.00       182
           3       0.56      0.35      0.43       286

    accuracy                           0.38       799
   macro avg       0.22      0.29      0.23       799
weighted avg       0.31      0.38      0.31     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch 2/30
Training Loss: 2.3829
Validation Loss: 2.2759
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       140
           1       0.65      0.74      0.70       274
           2       0.00      0.00      0.00        99
           3       0.49      0.84      0.62       286

    accuracy                           0.56       799
   macro avg       0.29      0.40      0.33       799
weighted avg       0.40      0.56      0.46       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        74
           1       0.41      0.19      0.26       257
           2       0.45      0.64      0.53       182
           3       0.52      0.77      0.62       286

    accuracy                           0.48       799
   macro avg       0.34      0.40      0.35       799
weighted avg       0.42      0.48      0.42     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

...

Epoch 26/30
Training Loss: 1.3985
Validation Loss: 2.4100
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.36      0.49      0.41       140
           1       0.78      0.74      0.76       274
           2       0.49      0.42      0.46        99
           3       0.60      0.56      0.58       286

    accuracy                           0.59       799
   macro avg       0.56      0.55      0.55       799
weighted avg       0.61      0.59      0.60       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.46      0.42      0.44        74
           1       0.48      0.42      0.45       257
           2       0.52      0.65      0.58       182
           3       0.59      0.56      0.57       286

    accuracy                           0.52       799
   macro avg       0.51      0.51      0.51       799
weighted avg       0.52      0.52      0.52    

Epoch 27/30
Training Loss: 1.3989
Validation Loss: 2.4012
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.36      0.49      0.41       140
           1       0.78      0.77      0.78       274
           2       0.52      0.46      0.49        99
           3       0.62      0.54      0.58       286

    accuracy                           0.60       799
   macro avg       0.57      0.57      0.56       799
weighted avg       0.62      0.60      0.61       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.45      0.45      0.45        74
           1       0.47      0.42      0.44       257
           2       0.51      0.66      0.57       182
           3       0.60      0.54      0.57       286

    accuracy                           0.52       799
   macro avg       0.51      0.52      0.51       799
weighted avg       0.52      0.52      0.52    

Epoch 28/30
Training Loss: 1.4094
Validation Loss: 2.4027
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.37      0.49      0.42       140
           1       0.78      0.76      0.77       274
           2       0.51      0.44      0.47        99
           3       0.62      0.55      0.58       286

    accuracy                           0.60       799
   macro avg       0.57      0.56      0.56       799
weighted avg       0.61      0.60      0.60       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.47      0.46      0.46        74
           1       0.48      0.42      0.45       257
           2       0.51      0.65      0.57       182
           3       0.60      0.55      0.58       286

    accuracy                           0.53       799
   macro avg       0.51      0.52      0.51       799
weighted avg       0.53      0.53      0.52    

Epoch 29/30
Training Loss: 1.3843
Validation Loss: 2.4043
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.37      0.49      0.42       140
           1       0.78      0.76      0.77       274
           2       0.51      0.44      0.47        99
           3       0.62      0.55      0.58       286

    accuracy                           0.60       799
   macro avg       0.57      0.56      0.56       799
weighted avg       0.61      0.60      0.60       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        74
           1       0.47      0.42      0.44       257
           2       0.51      0.65      0.57       182
           3       0.60      0.55      0.57       286

    accuracy                           0.52       799
   macro avg       0.51      0.52      0.51       799
weighted avg       0.52      0.52      0.52    

Epoch 30/30
Training Loss: 1.3968
Validation Loss: 2.4047
Classification Report for Target Task:
              precision    recall  f1-score   support

           0       0.37      0.49      0.42       140
           1       0.78      0.77      0.77       274
           2       0.51      0.44      0.47        99
           3       0.62      0.55      0.59       286

    accuracy                           0.60       799
   macro avg       0.57      0.56      0.56       799
weighted avg       0.62      0.60      0.61       799

Classification Report for Severity Task:
              precision    recall  f1-score   support

           0       0.45      0.45      0.45        74
           1       0.47      0.42      0.45       257
           2       0.51      0.65      0.57       182
           3       0.59      0.55      0.57       286

    accuracy                           0.52       799
   macro avg       0.51      0.52      0.51       799
weighted avg       0.52      0.52      0.52    

generate predictions

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm  # For progress bars
from sklearn.preprocessing import LabelEncoder

# Custom Dataset for Test Data
class TargetSeverityDatasetTest(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Multi-task Model Definition
class MultiTaskTargetSeverityClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.3, num_target_classes=4, num_severity_classes=4):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.target_classifier = nn.Linear(256, num_target_classes)
        self.severity_classifier = nn.Linear(256, num_severity_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        target_output = F.softmax(self.target_classifier(x), dim=1)
        severity_output = F.softmax(self.severity_classifier(x), dim=1)

        return target_output, severity_output

# Load the saved model
def load_model(model_path, device):
    tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")  # Use hateBERT tokenizer
    bert_model = AutoModel.from_pretrained("GroNLP/hateBERT")  # Use hateBERT model
    model = MultiTaskTargetSeverityClassifier(bert_model).to(device)
    model.load_state_dict(torch.load(model_path))  # Load trained model weights
    model.eval()
    return model, tokenizer

# Prediction function
def make_predictions(model, tokenizer, test_loader, device):
    all_predictions = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Making Predictions", leave=False, ncols=100, dynamic_ncols=True):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target_output, severity_output = model(input_ids, attention_mask)

            # Get predicted classes (indices with max probability)
            target_preds = torch.argmax(target_output, dim=1).cpu().numpy()
            severity_preds = torch.argmax(severity_output, dim=1).cpu().numpy()

            all_predictions.extend(zip(target_preds, severity_preds))
    return all_predictions

# Main function for making predictions and saving to CSV
def main():
    # Load the test data
    df_test = pd.read_csv(r'/content/drive/MyDrive/Test_trans.csv')  # Replace with your test data path

    # Initialize tokenizer and device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the model
    model, tokenizer = load_model(r'/content/drive/MyDrive/best_model.pth', device)  # Replace with your model path

    # Create the test dataset and DataLoader
    test_dataset = TargetSeverityDatasetTest(df_test['Tweet'].values, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=32)

    # Make predictions
    predictions = make_predictions(model, tokenizer, test_loader, device)

    # Initialize LabelEncoders and mappings
    target_mapping = ['I', 'O', 'R', 'N/A']
    severity_mapping = ['H', 'L', 'M', 'N/A']

    # Map predictions back to original labels
    df_test['Predicted_Target'] = [target_mapping[pred[0]] for pred in predictions]
    df_test['Predicted_Severity'] = [severity_mapping[pred[1]] for pred in predictions]

    # Save predictions to CSV
    df_test[['Id', 'Predicted_Target', 'Predicted_Severity']].to_csv(r'/content/drive/MyDrive/taskB_hate_predictions.csv', index=False)
    print("Predictions saved to 'taskB_predictions.csv'.")

# Run the main function
if __name__ == '__main__':
    main()


<ipython-input-4-3944e23f9d58>:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load trained model weights
                

Predictions saved to 'taskB_predictions.csv'.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
